In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

#Going to try going in through the API in order to grab as much relevant data as possible
import json
import requests
from pprint import pprint

In [4]:
#Outer-most layer
base_url = "https://chicagohealthatlas.org/api/v1/"
outer_layer = requests.get(base_url).json()
pprint(outer_layer)

{'Atlas': 'https://chicagohealthatlas.org/api/v1/atlas/',
 'Categories': 'https://chicagohealthatlas.org/api/v1/categories/',
 'Colors': 'https://chicagohealthatlas.org/api/v1/colors/',
 'Data': 'https://chicagohealthatlas.org/api/v1/data/',
 'Data Coverage': 'https://chicagohealthatlas.org/api/v1/coverage/',
 'Datasets': 'https://chicagohealthatlas.org/api/v1/datasets/',
 'Geographic layers': 'https://chicagohealthatlas.org/api/v1/layers/',
 'Geographies': 'https://chicagohealthatlas.org/api/v1/geographies/',
 'Periods': 'https://chicagohealthatlas.org/api/v1/periods/',
 'Regions': 'https://chicagohealthatlas.org/api/v1/regions/',
 'Stratification Groupings': 'https://chicagohealthatlas.org/api/v1/population-groupings/',
 'Stratifications': 'https://chicagohealthatlas.org/api/v1/populations/',
 'Terms': 'https://chicagohealthatlas.org/api/v1/terms/',
 'Topics': 'https://chicagohealthatlas.org/api/v1/topics/'}


In [3]:
#Atlas
atlas_url = outer_layer['Atlas']

In [37]:
#Categories [1/...]
categories_url = outer_layer['Categories']
categories_json = requests.get(categories_url).json()

# Turned some of the data from categories_json into lists...
categories_list=["Clinical%20Care", "Physical%20Environment","Morbidity","Social%20and%20Economic%20Factors",
                "Health%20Behaviors","Demography","Mortality"]
subcategories_list=['access-to-care','quality-of-care','community-safety-1','housing-transit','pollution',
                    'resource-availability','behavioral-health','chronic-disease-1','infectious-disease-1',
                   'injury-violence-1','maternal-infant-child-adolescent-health','quality-of-life','community-safety',
                   'education','employment','family-social-support','income','public-assistance','alcohol-drug-use',
                   'diet-exercise','sexual-activity','tobacco-use','origin-language','population','behavioral-health-1',
                   'chronic-disease','infectious-disease','injury-violence','maternal-infant-child-adolescent-health-1',
                   'quality-of-life-1']

# Would like to build network/matrix of subcategory relations...

subcat_rels_dict = {}
for category in categories_list:
    subcat_rels_dict[category]=list()


subcat_count=0
for subcategory in subcategories_list:
    cat_count=0
    for category in categories_list:
        local_subcats=[]
        for i in range(len(categories_json['results'][cat_count]['subcategories'])):
            local_subcats.append(categories_json['results'][cat_count]['subcategories'][i]['slug'])
        if subcategory in local_subcats:
            subcat_rels_dict[category].append(True)
        else:
            subcat_rels_dict[category].append(False)
        cat_count += 1
    subcat_count += 1
# subcat_rels_dict
    
subcat_rels_df = pd.DataFrame(subcat_rels_dict)
subcat_rels_df[""]=subcategories_list
subcat_rels_df = subcat_rels_df.set_index("") #Notice this is a block-diagonal matrix!

is_subcategory = lambda x,y: subcat_rels_df.loc[x][y]
is_subcategory('access-to-care','Morbidity')

False

In [36]:
subcat_rels_df.loc['access-to-care']['Morbidity']

False

In [38]:
#Topics [1/...]
topics_url = outer_layer['Topics']
topics_json = requests.get(topics_url).json()

# pprint(topics_json) #There are 284 topics

# topic_names_list = list()
# for x in range(284):
#     topic_names_list.append(topics_json['results'][x]['name'])
# topic_names_list

# Default response is limited to 20 results, need to set limit with parameter...

all_topics_json = requests.get(topics_url + '?limit=284').json()
topic_names_list = list()
for x in range(284):
       topic_names_list.append(all_topics_json['results'][x]['name'])
topic_keys_list = []
for x in range(284):
       topic_keys_list.append(all_topics_json['results'][x]['key'])

topics_dict={}        
for x in range(284):
    topics_dict[topic_keys_list[x]]=topic_names_list[x]
topics_dict

{'VRAC': 'Accidents mortality',
 'VRACR': 'Accidents mortality rate',
 'HCSATH': 'Adult asthma',
 'HCSATHP': 'Adult asthma rate',
 'HCSBD': 'Adult binge drinking',
 'HCSBDP': 'Adult binge drinking rate',
 'HCSDIA': 'Adult diabetes',
 'HCSDIAP': 'Adult diabetes rate',
 'HCSEC': 'Adult e-cigarette use',
 'HCSECP': 'Adult e-cigarette use rate',
 'HCSFLU': 'Adult flu vaccination',
 'HCSFLUP': 'Adult flu vaccination rate',
 'HCSFV': 'Adult fruit and vegetable servings',
 'HCSFVP': 'Adult fruit and vegetable servings rate',
 'HCSOB': 'Adult obesity',
 'HCSOBP': 'Adult obesity rate',
 'HCSPA': 'Adult physical inactivity',
 'HCSPAP': 'Adult physical inactivity rate',
 'HCSSMK': 'Adult smoking',
 'HCSSMKP': 'Adult smoking rate',
 'HCSS': 'Adult soda consumption',
 'HCSSP': 'Adult soda consumption rate',
 'CZA': 'Aggravated assault/battery',
 'VRAL': 'Alcohol-induced mortality',
 'VRALR': 'Alcohol-induced mortality rate',
 'HDALR': 'Alcohol-related hospitalization rate',
 'HDAL': 'Alcohol-relate

In [39]:
pprint(all_topics_json)

{'count': 284,
 'next': None,
 'previous': None,
 'results': [{'datasets': [{'dataset': {'label': 'death-certificate-file',
                                        'name': 'Illinois Department of Public '
                                                'Health, Death Certificate '
                                                'Data Files'},
                            'notes': ''}],
              'description': 'Number of people who died due to an '
                             'unintentional injury.',
              'direction': 'Neutral',
              'format': '0 decimal places',
              'is_count': True,
              'key': 'VRAC',
              'keywords': 'Accidental, unintentional injury',
              'name': 'Accidents mortality',
              'restrictions': [],
              'subcategories': [{'category': 'Mortality',
                                 'name': 'Injury & Violence',
                                 'slug': 'injury-violence'}],
              'units': '

                                        'name': 'Illinois Department of Public '
                                                'Health, Death Certificate '
                                                'Data Files'},
                            'notes': ''}],
              'description': 'Number of deaths due to all causes.',
              'direction': 'Bad',
              'format': '0 decimal places',
              'is_count': True,
              'key': 'VRDTH',
              'keywords': 'total deaths, number of deaths',
              'name': 'All-cause mortality',
              'restrictions': [],
              'subcategories': [{'category': 'Mortality',
                                 'name': 'Chronic Disease',
                                 'slug': 'chronic-disease'}],
              'units': 'count of deaths'},
             {'datasets': [{'dataset': {'label': 'death-certificate-file',
                                        'name': 'Illinois Department of Public '
          

                                        'name': 'Illinois Department of Public '
                                                'Health, Illinois’ National '
                                                'Electronic Disease '
                                                'Surveillance System '
                                                '(I-NEDSS)'},
                            'notes': ''}],
              'description': 'Individuals diagnosed with chlamydia infection.',
              'direction': 'Bad',
              'format': '0 decimal places',
              'is_count': True,
              'key': 'STDCH',
              'keywords': 'STI, sexually transmitted infection, STD, sexually '
                          'transmitted disease',
              'name': 'Chlamydia',
              'restrictions': [],
              'subcategories': [{'category': 'Morbidity',
                                 'name': 'Infectious Disease',
                                 'slug': 'infectious-dis

              'direction': 'Bad',
              'format': '2 decimal places',
              'is_count': False,
              'key': 'DIS',
              'keywords': 'disabled,handicapped,impaired',
              'name': 'Disability',
              'restrictions': [],
              'subcategories': [{'category': 'Morbidity',
                                 'name': 'Quality of Life',
                                 'slug': 'quality-of-life'}],
              'units': '% of residents'},
             {'datasets': [{'dataset': {'label': 'idph-cancer',
                                        'name': 'Illinois Department of Public '
                                                'Health, Illinois State Cancer '
                                                'Registry'},
                            'notes': 'Calculated by Metopio'}],
              'description': 'Annual diagnosis rate for cancers with distant '
                             'or systemic spread, often referred to as Stage 4 '

              'description': 'Number of people who died due to heart disease.',
              'direction': 'Neutral',
              'format': '0 decimal places',
              'is_count': True,
              'key': 'VRHD',
              'keywords': None,
              'name': 'Heart disease mortality',
              'restrictions': [],
              'subcategories': [{'category': 'Mortality',
                                 'name': 'Chronic Disease',
                                 'slug': 'chronic-disease'}],
              'units': 'count of deaths'},
             {'datasets': [{'dataset': {'label': 'death-certificate-file',
                                        'name': 'Illinois Department of Public '
                                                'Health, Death Certificate '
                                                'Data Files'},
                            'notes': ''}],
              'description': 'Age-adjusted rate of people who died due to '
                        

              'name': 'Mood and depressive disorder hospitalizations',
              'restrictions': [],
              'subcategories': [{'category': 'Morbidity',
                                 'name': 'Behavioral Health',
                                 'slug': 'behavioral-health'}],
              'units': 'count of hospitalizations'},
             {'datasets': [{'dataset': {'label': 'death-certificate-file',
                                        'name': 'Illinois Department of Public '
                                                'Health, Death Certificate '
                                                'Data Files'},
                            'notes': ''}],
              'description': 'Number of people who died due to a motor vehicle '
                             'crash.',
              'direction': 'Neutral',
              'format': '0 decimal places',
              'is_count': True,
              'key': 'VRMV',
              'keywords': None,
              'name': 'M

                            'notes': ''}],
              'description': 'Percent of adults who report that they have at '
                             'least one person they think of as their personal '
                             'doctor or health care provider.',
              'direction': 'Good',
              'format': '1 decimal place',
              'is_count': False,
              'key': 'HCSPCPP',
              'keywords': None,
              'name': 'Primary care provider rate',
              'restrictions': [],
              'subcategories': [{'category': 'Clinical Care',
                                 'name': 'Access to Care',
                                 'slug': 'access-to-care'}],
              'units': '% of adults'},
             {'datasets': [{'dataset': {'label': 'crime-data-portal',
                                        'name': 'Crime data portal'},
                            'notes': ''}],
              'description': 'Property crimes. Includes burglary, la

                                 'name': 'Employment',
                                 'slug': 'employment'}],
              'units': '%'},
             {'datasets': [{'dataset': {'label': 'american-community-survey',
                                        'name': 'American Community Survey'},
                            'notes': 'Tables B27001/C27001'}],
              'description': 'Percent of residents without health insurance '
                             '(at the time of the survey).',
              'direction': 'Bad',
              'format': '2 decimal places',
              'is_count': False,
              'key': 'UNS',
              'keywords': 'self pay, insurance, insured, coverage',
              'name': 'Uninsured rate',
              'restrictions': [],
              'subcategories': [{'category': 'Clinical Care',
                                 'name': 'Access to Care',
                                 'slug': 'access-to-care'}],
              'units': '% of residents

In [41]:
#Topics cont'd [2/...]
#
# Would like to associate topics with their subcategories, for at least a couple categories...
# So something like "is_subcategory" above, but instead "has_subcategory"...

topic_subcat_rels_dict = {}
for subcategory in subcategories_list:
    topic_subcat_rels_dict[subcategory]=list()


topic_count=0
for topic in topic_keys_list:
    local_subcats=[]
    for i in range(len(all_topics_json['results'][topic_count]['subcategories'])):
        local_subcats.append(all_topics_json['results'][topic_count]['subcategories'][i]['slug'])
    subcat_count=0
    for subcategory in subcategories_list:
        if subcategory in local_subcats:
            topic_subcat_rels_dict[subcategory].append(True)
        else:
            topic_subcat_rels_dict[subcategory].append(False)
        subcat_count += 1
    topic_count += 1
# topic_subcat_rels_dict
    
topic_subcat_rels_df = pd.DataFrame(topic_subcat_rels_dict)
topic_subcat_rels_df[""]=topic_keys_list
topic_subcat_rels_df = topic_subcat_rels_df.set_index("") 

topic_subcat_rels_df

,access-to-care,quality-of-care,community-safety-1,housing-transit,pollution,resource-availability,behavioral-health,chronic-disease-1,infectious-disease-1,injury-violence-1,...,sexual-activity,tobacco-use,origin-language,population,behavioral-health-1,chronic-disease,infectious-disease,injury-violence,maternal-infant-child-adolescent-health-1,quality-of-life-1
,,,,,,,,,,,,,,,,,,,,,
VRAC,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
VRACR,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
HCSATH,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
HCSATHP,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
HCSBD,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YRSMKP,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
YRSO,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
YRSOP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [42]:
#Topics cont'd [3/...]
#
# Turn dataframe into a lambda function...

has_subcategory = lambda x,y: topic_subcat_rels_df.loc[x][y]
has_subcategory('VRAC','injury-violence')

True

In [5]:
#Colors

In [7]:
#Data Coverage [1/...]
data_cov_url = outer_layer['Data Coverage']
# data_cov_json = requests.get(data_cov_url).json()
# pprint(data_cov_json) #Response: {'detail': 'No topic key provided.'}
# Need to use endpoint URL /api/v1/coverage/<topic_key>/

#Starting with topic 'Accidents mortality' with key 'VRAC'
acc_mort_url = data_cov_url + 'VRAC'
acc_mort_json = requests.get(acc_mort_url).json()
pprint(acc_mort_json)

{'coverages': {'neighborhood': [{'period': '2015-2019', 'population': ''},
                                {'period': '2014-2018', 'population': ''},
                                {'period': '2013-2017', 'population': ''},
                                {'period': '2012-2016', 'population': ''},
                                {'period': '2011-2015', 'population': ''},
                                {'period': '2010-2014', 'population': ''},
                                {'period': '2005-2009', 'population': ''},
                                {'period': '2000-2004', 'population': ''}],
               'place': [{'period': '2020', 'population': ''},
                         {'period': '2015-2019', 'population': ''},
                         {'period': '2019', 'population': ''},
                         {'period': '2014-2018', 'population': ''},
                         {'period': '2018', 'population': ''},
                         {'period': '2017', 'population': ''},
           

In [8]:
#Data coverage (cont'd) [2/...]

#Looking at 'Presciption opiate misuse' with key 'HCSOP' to see one that only has data from a few neighborhoods
op_mis_url = data_cov_url + 'HCSOP'
op_mis_json = requests.get(op_mis_url).json()
pprint(op_mis_json)

{'coverages': {'neighborhood': [{'period': '2020-2021', 'population': ''}],
               'place': [{'period': '2020-2021', 'population': ''},
                         {'period': '2021', 'population': ''},
                         {'period': '2020', 'population': ''},
                         {'period': '2018', 'population': ''},
                         {'period': '2017', 'population': ''},
                         {'period': '2016', 'population': ''},
                         {'period': '2015', 'population': ''},
                         {'period': '2021', 'population': 'F'},
                         {'period': '2020', 'population': 'F'},
                         {'period': '2018', 'population': 'F'},
                         {'period': '2017', 'population': 'F'},
                         {'period': '2016', 'population': 'F'},
                         {'period': '2015', 'population': 'F'},
                         {'period': '2021', 'population': 'M'},
                         {'per

In [9]:
#Data coverage (cont'd) [3/...]
#
# Ok, that doesn't look any different, so if we want to see the neighborhood coverage we'll need to get that data from
# somewhere else

# Let's first look at one we know only has city-wide coverage...
# 'Youth binge drinking' with key 'YRBD'
y_drink_url = data_cov_url + 'YRBD'
y_drink_json = requests.get(y_drink_url).json()
pprint(y_drink_json)

{'coverages': {'neighborhood': [],
               'place': [{'period': '2019', 'population': ''},
                         {'period': '2017', 'population': ''},
                         {'period': '2019', 'population': 'F'},
                         {'period': '2017', 'population': 'F'},
                         {'period': '2019', 'population': 'M'},
                         {'period': '2017', 'population': 'M'},
                         {'period': '2019', 'population': 'W'},
                         {'period': '2017', 'population': 'W'},
                         {'period': '2017', 'population': 'B'},
                         {'period': '2019', 'population': 'H'},
                         {'period': '2017', 'population': 'H'},
                         {'period': '2019', 'population': 'E'},
                         {'period': '2017', 'population': 'E'},
                         {'period': '2019', 'population': 'L'},
                         {'period': '2017', 'population': 'L'}],
      

In [9]:
#Data coverage (cont'd) [4/...]
#
# Yeah, that gives an empty result for 'neighborhood.'
#
# Going to try to automate the creation of a bar chat plotting the number of indicators covered for each year...
# First, copy the topics dictionary from below to use keys in automated search
topics_url = outer_layer['Topics']
all_topics_json = requests.get(topics_url + '?limit=284').json()
topic_names_list = list()
for x in range(284):
       topic_names_list.append(all_topics_json['results'][x]['name'])
topic_keys_list = []
for x in range(284):
       topic_keys_list.append(all_topics_json['results'][x]['key'])

topics_dict={}        
for x in range(284):
    topics_dict[topic_keys_list[x]]=topic_names_list[x]
topics_dict

{'VRAC': 'Accidents mortality',
 'VRACR': 'Accidents mortality rate',
 'HCSATH': 'Adult asthma',
 'HCSATHP': 'Adult asthma rate',
 'HCSBD': 'Adult binge drinking',
 'HCSBDP': 'Adult binge drinking rate',
 'HCSDIA': 'Adult diabetes',
 'HCSDIAP': 'Adult diabetes rate',
 'HCSEC': 'Adult e-cigarette use',
 'HCSECP': 'Adult e-cigarette use rate',
 'HCSFLU': 'Adult flu vaccination',
 'HCSFLUP': 'Adult flu vaccination rate',
 'HCSFV': 'Adult fruit and vegetable servings',
 'HCSFVP': 'Adult fruit and vegetable servings rate',
 'HCSOB': 'Adult obesity',
 'HCSOBP': 'Adult obesity rate',
 'HCSPA': 'Adult physical inactivity',
 'HCSPAP': 'Adult physical inactivity rate',
 'HCSSMK': 'Adult smoking',
 'HCSSMKP': 'Adult smoking rate',
 'HCSS': 'Adult soda consumption',
 'HCSSP': 'Adult soda consumption rate',
 'CZA': 'Aggravated assault/battery',
 'VRAL': 'Alcohol-induced mortality',
 'VRALR': 'Alcohol-induced mortality rate',
 'HDALR': 'Alcohol-related hospitalization rate',
 'HDAL': 'Alcohol-relate

In [11]:
#Data coverage (cont'd) [5/...]
#
# Create period coverage dictionary
period_cov_dict={}

# Loop over keys in topics dictionary  
for key in topics_dict:
    topic_cov_url = data_cov_url + key
    topic_cov_json = requests.get(topic_cov_url).json()
    for x in range(len(topic_cov_json['coverages']['place'])):
        if topic_cov_json['coverages']['place'][x]['population'] == '':
            if period_cov_dict.get(topic_cov_json['coverages']['place'][x]['period']) == None:
                period_cov_dict[topic_cov_json['coverages']['place'][x]['period']] = 1
            else:
                period_cov_dict[topic_cov_json['coverages']['place'][x]['period']] = \
                    period_cov_dict.get(topic_cov_json['coverages']['place'][x]['period']) + 1

period_cov_dict

{'2020': 139,
 '2015-2019': 94,
 '2019': 150,
 '2014-2018': 96,
 '2018': 168,
 '2017': 225,
 '2016': 203,
 '2015': 184,
 '2014': 175,
 '2013': 161,
 '2012': 140,
 '2011': 158,
 '2010': 131,
 '2009': 107,
 '2008': 87,
 '2007': 105,
 '2006': 87,
 '2005': 103,
 '2004': 87,
 '2003': 101,
 '2002': 87,
 '2001': 101,
 '2000': 87,
 '2020-2021': 52,
 '2021': 68,
 '2017-2021': 9,
 '2016-2020': 39,
 '2013-2017': 32,
 '2012-2016': 29,
 '2011-2015': 34,
 '2010-2014': 31,
 '2009-2013': 34,
 '2008-2012': 31,
 '2007-2011': 24,
 '2006-2010': 28,
 '2005-2009': 17,
 '2004-2008': 20,
 '2003-2007': 16,
 '2002-2006': 13,
 '2001-2005': 16,
 '2000-2004': 13,
 '1999-2003': 16,
 '1998-2002': 13,
 '1997-2001': 10,
 '1996-2000': 13,
 '1995-1999': 13,
 '1999': 12,
 '1994-1998': 17,
 '1998': 12,
 '1993-1997': 13,
 '1997': 12,
 '1992-1996': 10,
 '1996': 12,
 '1991-1995': 13,
 '1995': 10,
 '1990-1994': 13,
 '1994': 10,
 '1989-1993': 17,
 '1993': 10,
 '1988-1992': 13,
 '1992': 10,
 '1987-1991': 10,
 '1991': 10,
 '1986

In [12]:
#Data coverage (cont'd) [6/...]
#
# Wow! That worked on the first try!
# Let's save that dictionary to a csv, it took a while to run through all the API calls...
# First turn dictionary into a dataframe
col_1 = list(period_cov_dict.keys())
col_2 = [period_cov_dict[key] for key in col_1]
data = {'Year/Range': col_1, 'Indicator Count': col_2}
period_cov_df = pd.DataFrame({'Year/Range': col_1, 'Indicator Count': col_2})
period_cov_df

,Year/Range,Indicator Count
0,2020,139
1,2015-2019,94
2,2019,150
3,2014-2018,96
4,2018,168
...,...,...
113,2018.05,3
114,2018.04,3
115,2018.03,3
116,2018.02,3


In [13]:
#Data coverage (cont'd) [7/...]
#
# Now save df to csv
csv_file = "Resources/PeriodCoverage.csv"
period_cov_df.to_csv(csv_file, index = False, header=True)

In [5]:
#Periods
#
# Let's actually study the periods first...

periods_url = outer_layer['Periods']
periods_json = requests.get(periods_url).json()
pprint(periods_json)

{'count': 786,
 'next': 'https://chicagohealthatlas.org/api/v1/periods/?limit=20&offset=20',
 'previous': None,
 'results': [{'display': '2100',
              'duration': '1 year',
              'end_month': None,
              'end_year': 2100,
              'start_month': None,
              'start_year': 2100},
             {'display': '2029',
              'duration': '1 year',
              'end_month': None,
              'end_year': 2029,
              'start_month': None,
              'start_year': 2029},
             {'display': '2021-2025',
              'duration': '5 years',
              'end_month': None,
              'end_year': 2025,
              'start_month': None,
              'start_year': 2021},
             {'display': '2025',
              'duration': '1 year',
              'end_month': None,
              'end_year': 2025,
              'start_month': None,
              'start_year': 2025},
             {'display': '2020-2024',
              'duration': '5

In [10]:
#Periods (cont'd) [2/...]
#
# Let's look at the results for the year 2100

y2100_url = periods_url + f'/2100/'
y2100_json = requests.get(y2100_url).json()
#pprint(y2100_json) #Ok so there's nothing much... does any topic key give us a hit for 2100?

In [49]:
#Data
#
# Want to define the indicators (topics) as functions from time and space to data...
# What are all the functions that take indicators?
#
# Start with the year 2100...

data_url = outer_layer['Data']

# y2100_dict = {}
# for key in topic_keys_list:
#     key2100_url = data_url + f'?topic={key}&population=""&period=2100'
#     key2100_json = requests.get(key2100_url).json()
#     y2100_dict[key]=key2100_json
    
# y2100_dict # Aw, no results there yet... maybe we can work on predicting that someday...

# We can instead focus on the years 2000-2020...
# Fundamental question: what changes have the strongest correlation? 
# We focus on difference because we believe that all values are produced by gaps in our data, or "blindspots".
# That is to say, no simple functional relations between our data exist, the data is produced by sources for which we 
# do not have data.
# We do not expect the values of our data to be directly influencing each other, but we expect the relations between 
# their changes to tell us something about the source of these changes.

# First of all, what years do we even have good population data for?
# POP data starts in 2010, well-stratified (count=939).
# So focus instead on 2010-2020...

years = range(2010,2021)

data_dict = {}
for key in topic_keys_list:
    data_dict[key]=list()
    
for key in topic_keys_list:
    for year in years:
        key_year_url = data_url + f'?topic={key}&population=&period={year}'
        key_year_json = requests.get(key_year_url).json()
        try:
            key_year_value = key_year_json['results'][0]['v']
            data_dict[key].append(key_year_value)
        except IndexError:
            pass

data_df = pd.DataFrame(data_dict)

ValueError: arrays must all be same length

In [50]:
data_dict

{'VRAC': [670.0,
  636.0,
  791.0,
  816.0,
  790.0,
  824.0,
  1127.0,
  1255.0,
  1295.0,
  1320.0,
  1831.0],
 'VRACR': [25.5, 24.7, 29.9, 31.1, 30.0, 31.2, 42.4, 47.4, 49.0, 50.0, 69.0],
 'HCSATH': [174000.0, 168000.0, 216000.0, 188000.0, 204000.0, 197000.0],
 'HCSATHP': [9.1, 8.2, 10.5, 9.1, 9.5, 9.4480728334],
 'HCSBD': [515000.0, 496000.0, 505000.0, 553000.0, 695000.0],
 'HCSBDP': [25.4, 24.2, 24.8, 26.4, 34.532906676],
 'HCSDIA': [173000.0, 194000.0, 220000.0, 188000.0, 212000.0, 254000.0],
 'HCSDIAP': [9.0, 9.4, 10.6, 9.1, 9.9, 12.045599207],
 'HCSEC': [75000.0, 87000.0, 62000.0, 54000.0, 105000.0, 86000.0],
 'HCSECP': [3.9, 4.2, 3.0, 2.6, 4.9, 4.1173965344],
 'HCSFLU': [711000.0],
 'HCSFLUP': [34.6],
 'HCSFV': [554000.0, 582000.0, 504000.0, 627000.0, 546000.0, 716000.0],
 'HCSFVP': [29.2, 28.4, 24.3, 30.5, 25.6, 34.298723177],
 'HCSOB': [556000.0, 605000.0, 617000.0, 637000.0, 662000.0, 671000.0],
 'HCSOBP': [29.0, 29.3, 29.7, 30.8, 30.8, 32.13051879],
 'HCSPA': [350000.0, 53

In [ ]:
# Want final step to be...
df.corr()

In [14]:
#Datasets

In [15]:
#Geographic layers

In [16]:
#Geographies

In [18]:
#Regions

In [19]:
#Stratification groupings

In [20]:
#Stratifications

In [21]:
#Terms

{'VRAC': 'Accidents mortality',
 'VRACR': 'Accidents mortality rate',
 'HCSATH': 'Adult asthma',
 'HCSATHP': 'Adult asthma rate',
 'HCSBD': 'Adult binge drinking',
 'HCSBDP': 'Adult binge drinking rate',
 'HCSDIA': 'Adult diabetes',
 'HCSDIAP': 'Adult diabetes rate',
 'HCSEC': 'Adult e-cigarette use',
 'HCSECP': 'Adult e-cigarette use rate',
 'HCSFLU': 'Adult flu vaccination',
 'HCSFLUP': 'Adult flu vaccination rate',
 'HCSFV': 'Adult fruit and vegetable servings',
 'HCSFVP': 'Adult fruit and vegetable servings rate',
 'HCSOB': 'Adult obesity',
 'HCSOBP': 'Adult obesity rate',
 'HCSPA': 'Adult physical inactivity',
 'HCSPAP': 'Adult physical inactivity rate',
 'HCSSMK': 'Adult smoking',
 'HCSSMKP': 'Adult smoking rate',
 'HCSS': 'Adult soda consumption',
 'HCSSP': 'Adult soda consumption rate',
 'CZA': 'Aggravated assault/battery',
 'VRAL': 'Alcohol-induced mortality',
 'VRALR': 'Alcohol-induced mortality rate',
 'HDALR': 'Alcohol-related hospitalization rate',
 'HDAL': 'Alcohol-relate